In [1]:
import requests
import json
import os
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML

In [2]:
# Custom JSON encoder to add colors
def colorize_json(obj, indent=0):
    if isinstance(obj, dict):
        items = []
        for key, value in obj.items():
            key_str = f'<span style="color: #F4D03F;">"{key}"</span>'
            value_str = colorize_json(value, indent + 4)
            items.append(f'{" " * indent}{key_str}: {value_str}')
        return "{\n" + ",\n".join(items) + f"\n{' ' * (indent - 4)}}}"
    elif isinstance(obj, list):
        items = [colorize_json(item, indent + 4) for item in obj]
        return "[\n" + ",\n".join(items) + f"\n{' ' * (indent - 4)}]"
    elif isinstance(obj, str):
        return f'<span style="color: #5DADE2;">"{obj}"</span>'
    elif isinstance(obj, (int, float)):
        return f'<span style="color: #AF7AC5;">{obj}</span>'
    elif obj is None:
        return '<span style="color: #E74C3C;">null</span>'
    elif isinstance(obj, bool):
        return f'<span style="color: #2ECC71;">{str(obj).lower()}</span>'
    else:
        return str(obj)

In [3]:
# API Gateway endpoint and API key
# original "https://85fnwlcuc2.execute-api.us-east-2.amazonaws.com/default/model-recommendations"
url = "https://api.netriasbdf.cloud/cde-recommendation"

api_key = "6P8lZuybOZ6d3tVnTt9lqJmJlXzrEwy6lkODijs9"

In [10]:
def load_csv_as_dict(csv_path: Path) -> dict:
    """Read a CSV and return dict-of-lists like `my_column_data`."""
    df = pd.read_csv(csv_path)
    return df.to_dict(orient="list")

In [12]:
# Put csv into json / dict strucutre
#data_dict = load_csv_as_dict("synthetic/tissue_comparison.csv")
data_dict = load_csv_as_dict("CHOP_data/SD_DYPMEHHF/Biospecimen.csv")
my_column_data = data_dict

In [4]:
# Columns to map to schema
#simple rna seq
my_column_data = {
    "sample_name": [
        "RNAseq_Control_1",
        "RNAseq_Control_2",
        "RNAseq_Drug_1",
        "RNAseq_Drug_2"
    ],
    "geo_id": [
        "GSM001",
        "GSM002",
        "GSM003",
        "GSM004"
    ],
    "sample_type": [
        "tumor sample",
        "tumor sample",
        "tumor sample",
        "tumor sample"
    ],
    "species_name": [
        "Homo sapiens",
        "Homo sapiens",
        "Homo sapiens",
        "Homo sapiens"
    ],
    "tissue_source": [
        "breast tumor",
        "breast tumor",
        "breast tumor",
        "breast tumor"
    ],
    "treatment_condition": [
        "vehicle control",
        "vehicle control",
        "doxorubicin 50nM",
        "doxorubicin 50nM"
    ],
    "platform_used": [
        "Illumina HiSeq 2500",
        "Illumina HiSeq 2500",
        "Illumina HiSeq 2500",
        "Illumina HiSeq 2500"
    ],
    "sequencing_method": [
        "RNA-Seq",
        "RNA-Seq",
        "RNA-Seq",
        "RNA-Seq"
    ],
    "rna_source": [
        "total RNA",
        "total RNA",
        "total RNA",
        "total RNA"
    ],
    "prep_method": [
        "TruSeq",
        "TruSeq",
        "TruSeq",
        "TruSeq"
    ],
    "contact_lab": [
        "Smith Lab",
        "Smith Lab",
        "Smith Lab",
        "Smith Lab"
    ]
}


In [13]:
# Create and display the DataFrame
df = pd.DataFrame(my_column_data)
df

,uuid,created_at,modified_at,external_sample_id,external_aliquot_id,source_text_tissue_type,composition,source_text_anatomical_site,age_at_event_days,source_text_tumor_descriptor,...,duo_ids,visibility_reason,visibility_comment,preservation_method,sample_id,specimen_status,has_matched_normal_sample,amount,amount_units,cell_entity
0,83f066fc-3e88-4166-9af8-20c5e43a6506,2018-11-27 15:12:57.591143,2024-02-21 21:41:37.670392,GMKF-30-PAUAZW03-01A-01D,575541,Tumor,Solid Tissue,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_FPEX60NB,NaN,NaN,NaN,NaN,NaN
1,9d647002-c490-4d34-8838-79a4e364c651,2018-08-07 16:34:00.723260,2024-02-21 21:41:37.735410,GMKF-30-PAUMKS03-12A-01D,595355,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_V9AY2238,NaN,NaN,NaN,NaN,NaN
2,8caf5528-602f-4f25-ba18-4adba22771c7,2018-08-07 16:33:59.228235,2024-02-21 21:41:37.714636,GMKF-30-PAUPPZ03-12A-01D,595356,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_5WVQ70KP,NaN,NaN,NaN,NaN,NaN
3,bc9fab1b-0920-4d6c-927d-d625f92fec44,2018-11-27 15:13:02.273828,2024-02-21 21:41:37.666694,GMKF-30-PASJBY03-10A-01D,577084,Normal,Blood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_WDXX8AGG,NaN,NaN,NaN,NaN,NaN
4,14b9cadf-8dfe-4a6a-9aca-c30e4842e35a,2018-08-07 16:33:59.581710,2024-02-21 21:41:37.668530,GMKF-30-PAVCFM03-12A-01D,595394,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_FMXR3714,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152,b807c345-4699-4498-a9d3-51c960fdaa23,2018-09-06 21:26:03.352571,2024-02-21 21:42:11.324040,GMKF-30-PASBHS13-12A-01D,596716,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_4Y3SMT94,NaN,NaN,NaN,NaN,NaN
2153,7fdc4c1d-8277-4d9d-934d-e4c4187d0e10,2018-09-06 21:26:09.277003,2024-02-21 21:42:11.737670,GMKF-30-PAUXGR13-12A-01D,596770,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_PEWB6BKJ,NaN,NaN,NaN,NaN,NaN
2154,a9438480-2051-45d8-a14e-a14bbbb90485,2018-08-02 21:05:39.364770,2024-02-21 21:42:14.233015,GMKF-30-PARXPD03-12A-01D,595424,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_YC89WPNP,NaN,NaN,NaN,NaN,NaN
2155,1d79ee52-e368-44c0-b3fe-58a5d3f7027d,2018-09-06 21:27:02.736234,2024-02-21 21:42:14.316164,GMKF-30-PAUEKS06-12A-01D,596240,Normal,Buccal Cells,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SA_V0B6NZ57,NaN,NaN,NaN,NaN,NaN


In [ ]:
SCHEMAS = [
    "gc",
    "cds",
    "ccdi",
    "sage_chipseq",
    "sage_rnaseq",
    "sage_imagingassay",
    "sage_clinicalassay",
]

In [14]:
# Payload to send to Lambda (JSON string embedded in "body")
payload = {
    "body": json.dumps({
        "target_schema": "ccdi",
        "data": my_column_data
    })
}

In [15]:
# Headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": api_key
}


In [16]:
# Send POST request
response = requests.post(url, headers=headers, json=payload)


In [17]:
# Check the response and pretty print it
if response.status_code == 200:
    display(HTML("<p style='color: green;'>Request successful!</p>"))
    
    # Load the content as JSON
    response_content = response.json()
    
    # If 'body' is a string containing JSON, parse it
    if isinstance(response_content.get('body'), str):
        response_content['body'] = json.loads(response_content['body'])

    # Display the colorized JSON response
    display(HTML("<p>Response content:</p>"))
    display(HTML(f"<pre>{colorize_json(response_content)}</pre>"))
else:
    display(HTML(f"<p style='color: red;'>Request failed with status code: {response.status_code}</p>"))
    display(HTML(f"<p style='color: red;'>Response content:</p><pre>{response.text}</pre>"))